In [28]:
import numpy as np
import matplotlib.pyplot as plt
from natsort import natsorted
import os
import csv
import pandas as pd
from scipy.optimize import curve_fit
import glob
import pickle
from matplotlib.colors import LogNorm
from scipy.signal import find_peaks, savgol_filter

In [29]:
def label(x,y,a,b,pur="all"):
    plt.xlabel(x,fontsize=18)
    plt.ylabel(y,fontsize=18)
    if pur=="all":
        plt.rcParams['figure.figsize'] = [a,b]
    elif pur=="chi2":
        plt.title("Chi2 Analysis, S-%s, Channel %s"%(a,b), fontsize=20)
    plt.tick_params(axis='x', labelsize=14)
    plt.tick_params(axis='y', labelsize=14)
    return 

def gaus_fit(ri,rf,fx,fy,nm="norm"):
    from scipy.optimize import curve_fit
    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))
    from scipy.stats import chisquare
    b=len(bins)-1
    hist_PI=[]
    for j in range(b):
        if ri<=bins[j]<=rf:                                                          #Bins range for gauss fit
            hist_PI.extend([j])
    x=bins[hist_PI]+(bins[1]-bins[0])/2
    y=n[hist_PI]
    l = len(x)
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))#the number of data
    popt,pcov = curve_fit(gaus,x,y, p0=[max(y), mean, sigma])
    m=x[1]-x[0]
    X=np.arange(x.min(),x.max(),m/10)
#     chi2=chisquare(n[hist_PI],f_exp=gaus(x,*popt))
#     b=chi2[0]/(len(hist_PI)-3)
    if nm=="norm":
        textstr = '\n'.join((
        r'$\mu=%.2f \pm %.2f$ ADC' % (popt[1], np.sqrt(pcov[1,1])),
        r'$\sigma=%.2f \pm %.2f$ ADC' % (popt[2], np.sqrt(pcov[2,2]))
        ))               #"r'$\chi^2/Dof=%.2f$' % (b, )"
    elif nm=="keV":
        textstr = '\n'.join((
            r'$\mu=%.2f \pm %.1f$ keV' % (popt[1], np.sqrt(pcov[1,1])),
            r'$\sigma=%.2f \pm %.1f$ keV' % (popt[2], np.sqrt(pcov[2,2]))
            )) 
    elif nm=="keV_br":
        textstr = '\n'.join((
            r'$\mu=%.2f \pm %.2f$ eV' % (popt[1]*1000, np.sqrt(pcov[1,1])*1000),
            r'$\sigma=%.2f \pm %.2f$ eV' % (popt[2]*1000, np.sqrt(pcov[2,2])*1000)
            ))               #"r'$\chi^2/Dof=%.2f$' % (b, )"
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(fx, fy, textstr, fontsize=12,
             
             verticalalignment='top', bbox=props)
    return popt[0], popt[1],popt[2],plt.plot(X,gaus(X,*popt),'r')

In [31]:
N_det=26+11
Det_Comb=21

energyThreshold = 100
timeWindow=5e-7
Date="103024"
         #Not required if you are doing the full run. 
#first cut_data event or 2nd, Ex: index=0 means collecting first 60 second of data and 
                                 #index=1 means collecting second 60 second of data and so on...     
# run_index=0      #int(sys.argv[1])    #Type the index of run mentioned in the data file, required while taking multiple dataset 


activity = 1.47
cut_data= 100               #Select the required second of data
source = 'Ba133'
read_folder_all=glob.glob("E:/RUN5/Take5/Ba133/DAQ/CsI_Ba13*")                   #reading data folder
Total_folder = len(read_folder_all)
read_folder_all.sort(key=lambda x: os.stat(x).st_ctime)
folder_index = 0
read_folder= read_folder_all[folder_index]            #glob.glob("../RUN4/Sc46/DAQ/*")
save_folder="E:/RUN5/Take5/Ba133//Figures" 

print(f"Folder name {read_folder_all[folder_index]}")
index=0

innerChannel=np.arange(16,26)

date="%s"%(Date)
serial=np.array(["1st","2nd","3rd","4th","5th","6th", "7th", "8th"])
save_folder_each = os.path.join(save_folder, "Folder_%d"%folder_index)

Folder name E:/RUN5/Take5/Ba133/DAQ\CsI_Ba133


In [9]:
files = glob.glob('%s/RAW/*.CSV'%read_folder)
files=natsorted(files)[0:26]
# files = files
# print(files)
MinTime=np.zeros(N_det)
for nf,nfile in enumerate(files):
    # nfile1 = os.path.join(folder_path, nfile)
    file = open(nfile)
    csvreader = csv.reader(file, delimiter = ';')
    header = next(csvreader)
    line_count = 0
    for row in csvreader:
        if line_count == 0:
            MinTime[nf] = float(row[2]) * 1e-12
            break
minTime=np.min(MinTime)


# In[5]:

data=[]
for j in range(3):                                     #3=channel, time, calib. energy
    b=[]
    data.append(b)
nEvent=np.zeros(len(files))
for nf,nfile in enumerate(files):
    # nfile1 = os.path.join(folder_path, nfile)
    file = open(nfile)
    csvreader = csv.reader(file, delimiter = ';')
    header = next(csvreader)
    line_count = 0
    for row in csvreader:
        if float(row[2])*1e-12 - minTime>=cut_data*index:
            data[0].append(int(row[1]))
            data[1].append(float(row[2])*1e-12 - minTime)
            data[2].append(float(row[3]))     # row[4] = Caliberated energy, row[3] = ADC energy sometime, else row[3] = Caliberated energy
            line_count += 1
            # if line_count%1000000==0:
            #     print("%d M data loaded"%int(line_count/1000000))
            if float(row[2])*1e-12 - minTime>cut_data*(index+1):
                data[0].pop()
                data[1].pop()
                data[2].pop()
                break
    nEvent[nf]=line_count-1
    print("file %d done"%nf)
nEvents=int(nEvent.sum())



Data_Time=round((data[1][nEvents-1]-data[1][0]),0)
print("Total Time for this data set is %d s"%Data_Time)

file 0 done
file 1 done
file 2 done
file 3 done
file 4 done
file 5 done
file 6 done
file 7 done
file 8 done
file 9 done
file 10 done
file 11 done
file 12 done
file 13 done
file 14 done
file 15 done
file 16 done
file 17 done
file 18 done
file 19 done
file 20 done
file 21 done
file 22 done
file 23 done
file 24 done
file 25 done
Total Time for this data set is 100 s


In [10]:
nEvents = len(data[0])
energyPlot=[[] for _ in range(26)]
for i in range(nEvents):
    if data[0][i] < 26:
        energyPlot[int(data[0][i])].extend([data[2][i]])

In [11]:
mean = np.array([30.93, 45.15,47.66, 38.26, 52.63,     46.61, 40.73, 75.99, 39.51, 56.96,      32.3, 82.92, 74.96, 26.51, 90.15,
                25.1, 47.35, 44.57, 43.99, 53.14,     41.35, 42.5, 49.62, 24.81, 76.86, 105.77])
Mean = np.zeros(26)

In [26]:
def find_1peaks(x, y, channel, window_length, folder):
    from scipy.signal import find_peaks
    from scipy.signal import find_peaks, savgol_filter
    y_smooth = savgol_filter(y, window_length, polyorder=3)
    
    # Find the minimum value of the spectrum
    min_value = np.min(x)
    max_value = np.max(x)
    
    # Define the threshold as 20% of the minimum value
    threshold = 0.4 * max_value + min_value 
    
    # Create a mask to select the data points where y is greater than 20% of the minimum value
    mask = x >= threshold
    
    # Subset the data starting from the point where the values are above the threshold
    x_subset = x[mask]
    y_subset = y_smooth[mask]
    
    # Find peaks in the subset of the data
    peaks, _ = find_peaks(y_subset, distance=10, height=0)  # Adjust distance to ensure separation
    
    # If no peaks are found, you can handle it accordingly
    if len(peaks) > 0:
        # Get the index of the single highest peak in the subset
        top_peak_idx = np.argmax(y_subset[peaks])
        peak_x = x_subset[peaks[top_peak_idx]]
        peak_y = y_subset[peaks[top_peak_idx]]
    else:
        peak_x = None
        peak_y = None
    
    # Print the results
    print(f"Peak found at x = {peak_x}, y = {peak_y}")

    
    textstr = r'$\mu=%d $ ADC' %(peak_x)                        #"r'$\chi^2/Dof=%.2f$' % (b, )"
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(int(peak_x), int(peak_y/2), textstr, fontsize=12,
             verticalalignment='top', bbox=props)
    
    # Plot the original data, smoothed data, and the detected peak
    plt.plot(x, y, label='Original Data', alpha=0.5)
    plt.plot(x, y_smooth, label='Smoothed Data', color='orange', linewidth=2)
    if peak_x is not None:
        plt.scatter(peak_x, peak_y, color='red', label='Detected Peak', zorder=5)
    plt.legend()
    # plt.yscale("log")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Peak Detection, channel %d'%channel)
    # plt.show()
    plt.savefig("%s/Energy_Ch%d.jpg"%(folder,channel))
    plt.close()
    return peak_x

In [27]:
for k in range(1):
    bin_last=mean[k]*2
    bin_start = 4 if mean[k] < 40 else 10
    bin_w = 1 if mean[k] < 40 else 2
    n,bins=np.histogram(energyPlot[k], bins=np.arange(bin_start,bin_last,bin_w))
    bin_centers = bins[1:]

    window_length = int(mean[k]/6)   #length of the window this and that side of mean
    # Mean[k, :] = find_peaks(bin_centers, n, k, window_length, min_distance, full_path)
    Mean[k] = find_1peaks(bin_centers, n, k, window_length, save_folder_each)
    
    print(f"fit channel {k} done, mean value found {Mean[k], Mean[k]}, given {mean[k], mean[k]}")

Peak found at x = 32.0, y = 50.028571428571446
fit channel 0 done, mean value found (32.0, 32.0), given (30.93, 30.93)


In [319]:
nEvents = len(data[0])
for j in range(nEvents):
    if data[0][j]<26:
        data[2][j]=data[2][j]/Mean[int(data[0][j])]*356
print("Calibration done")

Calibration done


In [320]:
Count_raw = np.zeros((200, 26))
for j in  range(nEvents):
    for index, E in enumerate(range(0, 2000, 10)):
        if E <= data[2][j]< E+10 and data[0][j]<30:
            Count_raw[index, data[0][j]]+= 1
    if j%1000000==0:
        print('%d M done'%int(j/1000000))

0 M done
1 M done
2 M done
3 M done
4 M done
5 M done
6 M done
7 M done
8 M done
9 M done
10 M done
11 M done
12 M done
13 M done
14 M done
15 M done
16 M done
17 M done
18 M done
19 M done
20 M done
21 M done
22 M done
23 M done
24 M done
25 M done


In [316]:
Count_data = np.zeros((200, 26))
for j in  range(nEvents):
    for index, E in enumerate(range(0, 2000, 10)):
        if E <= data[2][j]< E+10 and data[0][j]<30:
            Count_data[index, data[0][j]]+= 1
    if j%1000000==0:
        print('%d M done'%int(j/1000000))

0 M done
1 M done
2 M done
3 M done
4 M done
5 M done


IndexError: index 41 is out of bounds for axis 1 with size 26

In [321]:
def efficiency(z, a, b, c, d):
    return a*scipy.special.erf(d*z-c)+b

In [341]:
Count_data_new = np.array([sum(Count_data[i:i+5, c]) for i in range(30, 150)], dtype = int)
Count_raw_new = np.array([sum(Count_raw[i:i+5, c]) for i in range(30, 150)], dtype = int)

Count_data_new = np.concatenate((Count_data[0:30, c], Count_data_new))
Count_raw_new = np.concatenate((Count_raw[0:30, c], Count_raw_new))

In [349]:
len(Count_data[0:30, c])

30

In [350]:
popt_new = [[] for _ in range(26)]

for c in range(26):
    Count_data_new = np.array([sum(Count_data[i:i+5, c]) for i in range(30, 150, 5)], dtype = int)
    Count_raw_new = np.array([sum(Count_raw[i:i+5, c]) for i in range(30, 150, 5)], dtype = int)
    
    Count_data_new = np.concatenate((Count_data[0:30, c], Count_data_new))
    Count_raw_new = np.concatenate((Count_raw[0:30, c], Count_raw_new))
    Energy = np.concatenate((np.arange(0, 300, 10), np.arange(300, 1500, 50)))
    Efficiency = Count_data_new/Count_raw_new
    plt.plot(Energy, Efficiency, ".")
    
    # popt, pcov = curve_fit(efficiency, Energy, Efficiency)
    # popt_new[c] = popt
    # # plt.xlim(0, 250)
    # plt.plot(Energy, efficiency(Energy, *popt))
    label("Energy", "Sigma/Mu", 8, 5)
    plt.title(r"Channel %d"%(c), fontsize=16)
    plt.savefig("E:RUN5/Take5/Efficiency/Efficiency_Ch%d.jpg"%(c))
    plt.close()
    print("%d done"%c)

C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: invalid value encountered in divide
  Efficiency = Count_data_new/Count_raw_new


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done


C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: invalid value encountered in divide
  Efficiency = Count_data_new/Count_raw_new


12 done
13 done
14 done


C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: invalid value encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: invalid value encountered in divide
  Efficiency = Count_data_new/Count_raw_new


...

C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new


18 done
19 done
20 done
21 done


C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: invalid value encountered in divide
  Efficiency = Count_data_new/Count_raw_new


22 done
23 done
24 done
25 done


C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new
C:\Users\sharada\AppData\Local\Temp\ipykernel_6724\3986804037.py:10: RuntimeWarning: divide by zero encountered in divide
  Efficiency = Count_data_new/Count_raw_new


In [136]:
f = open('%s/DataInfo_Run.pickle'%save_folder, 'rb')
dat = pickle.load(f)


In [137]:
mean = dat['mean']

In [144]:
mean[:, 0] = np.array([80, 112, 130, 92, 84,   114, 106, 196, 90, 148,   80, 216, 184, 58, 224,    62, 116, 102, 100,136,     102, 102, 126, 62, 222, 238])

In [146]:
mean[:, 1] = mean[:,0]/889*1132

In [194]:
for k in range(26):
    bin_last=mean[k, 0]
    bin_start = 4 if mean[k, 0] < 40 else 10
    bin_w = 1 if mean[k, 0] < 40 else 2
    n,bins, patches=plt.hist(energyPlot[k], bins=np.arange(bin_start,bin_last,bin_w), histtype="step", label="%d events"%len(energyPlot[k]))
    label("Energy (ADC)", "Counts", 10,5)
    # plt.yscale("log")
    plt.title(r"Channel %d Energy Spectrum- 15 min data (Ba133)"%k, fontsize=16)
    plt.savefig("%s/Pulse/EnergyCal_Ch%d.jpg"%(save_folder_each,k))
    plt.close()

In [195]:
Mean, Sigma = np.zeros(26), np.zeros(26)

In [273]:
#### k=15
Energy_feature = [[] for _ in range(26)]

# mean = np.array([60, 86, 92, 72, 66,   88, 80, 145, 70, 110,    62, 158, 136, 46, 168,     48, 90, 82, 80, 102,     80, 80, 94, 48, 140, 194])

for k in [25]:
    bin_last =  mean[k, 0]*0.9
    bin_w=np.array([2,2,2,2,2,    2,2,2,2,2,    2,2,2,2,2,   1,2,2,2,2,  2,2,2,2,2,  2])
    nevent = len(energyPlot[k])
    n,bins, patches=plt.hist(energyPlot[k][0:int(nevent)], bins=np.arange(8,bin_last,bin_w[k]), histtype="step", label="%d events"%len(energyPlot[k]))
    label("Energy (ADC)", "Counts", 10,5)
    # plt.yscale("log")
    Energy_feature[k].extend([n])
    Energy_feature[k].extend([bins])
    plt.title(r"Channel %d Energy Spectrum- 15 min data (%s)"%(k, source), fontsize=16)
    factor1 = 2
    factor2 = 2
    
    a1, Mean[k],Sigma[k], fig = gaus_fit(94, 125, bin_last/3,max(n)-max(n)/10)
    # a2, Mean[k,1],Sigma[k,1], fig= gaus_fit(mean[k, 1] +2 - 16*factor1*0.65, mean[k, 1] + 18*factor2*0.65, bin_last/1.5, max(n)-max(n)/2)

    plt.legend(fontsize=14)
    plt.savefig("%s/Energy_Ch%d.jpg"%(save_folder_each,k))
    plt.close()
    print("%d done"%k)

25 done


In [274]:
Mean, Sigma

(array([ 30.92701316,  45.1473533 ,  47.65599373,  38.26195902,
         52.63034721,  46.61143786,  40.7303583 ,  75.9899937 ,
         39.50898796,  56.95601542,  32.29804916,  82.92430748,
         74.95977486,  26.51077623,  90.14599021,  25.10183644,
         47.34613994,  44.56665029,  43.99294687,  53.14485156,
         41.35306033,  42.49691108,  49.62066783,  24.81217013,
         76.86120128, 105.76922156]),
 array([ 4.5261761 ,  5.73649571,  7.07241681,  5.40266249,  6.85112378,
         5.13802255,  6.17700153,  8.32337681,  5.74389169,  6.76084945,
         4.35082554,  8.13540773,  8.61524416,  4.77394511, 10.40995344,
         3.02672871,  4.41207111,  6.17994709,  5.52371258,  6.95490601,
         5.0803467 ,  6.61969149,  5.28702531,  4.8839151 ,  9.79922041,
         9.58781654]))

In [51]:
sigma_mu_Cs = Sigma/Mean

In [131]:
sigma_mu_Na = Sigma/Mean

In [186]:
sigma_mu_Sc = Sigma/Mean

In [275]:
sigma_mu_Ba = Sigma/Mean

In [276]:
def resolution(N, c, d):
    return c + d /np.sqrt(N)

In [281]:
sigma_mu_Na[1, :]

array([0.11016266, 0.04727802])

In [293]:
Energy = np.array([356, 511, 667, 1132, 1275])
popt_new = [[] for _ in range(26)]

for c in range(26):
    Ratio = np.array([sigma_mu_Ba[c], sigma_mu_Na[c, 0], sigma_mu_Cs[c], sigma_mu_Sc[c, 1], sigma_mu_Na[c, 1]])
    plt.plot(Energy, Ratio, ".")

    popt, pcov = curve_fit(resolution, Energy, Ratio)
    popt_new[c] = popt
    Energy1 = np.linspace(350, 1300, 100)
    plt.plot(Energy1, resolution(Energy1, *popt))
    label("Energy", "Sigma/Mu", 8, 5)
    plt.title(r"Channel %d"%(c), fontsize=16)
    plt.savefig("E:RUN5/Take5/Sigma_Mu_Ch%d.jpg"%(c))
    plt.close()
    print("%d done"%c)

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done


In [300]:
popt_new = np.array(popt_new)

In [298]:
f = open('E:RUN5/Take5/Sigma_Mu_ratio_B07.pickle', 'wb')
Parameter={'sigma_mu_Cs':sigma_mu_Cs, 'sigma_mu_Na':sigma_mu_Na, 'sigma_mu_Ba':sigma_mu_Ba, 'sigma_mu_Sc':sigma_mu_Sc, 'Function': 'c + d /sqrt(N)',
          'Constants':popt_new, 'Energy':Energy}
pickle.dump(Parameter, f)
f.close()

In [299]:
Energy

array([ 356,  511,  667, 1132, 1275])

# efficiency

In [2]:
def search_and_sort_files(root_dir, partial_name):
    # Search for files in subdirectories with the partial name
    files = glob(f"{root_dir}/**/*{partial_name}*", recursive=True)
    
    # Get files with their modification times
    files_with_times = [(file, os.path.getmtime(file)) for file in files if os.path.isfile(file)]
    
    # Sort files by modification time (most recent first)
    sorted_files = sorted(files_with_times, key=lambda x: x[1])
    
    # Print sorted file paths with modification times
    # for file, mtime in sorted_files:
    #     print(f"{file} - Last Modified: {mtime}")
    
    return sorted_files

In [ ]:
root_directory="E:/RUN5/Take5/Ba133/Figures/Norm_Th"  
partial_name = "Efficiency_count"
sorted_files = search_and_sort_files(root_directory, partial_name)
len(sorted_files)

In [ ]:
file = sorted_files[0][0]
f = open('%s'%(file), 'rb')
dat=pickle.load(f)
Count = dat['Count']
Energy_range=dat['Energy_range']
np.shape(Count)

In [ ]:

Count_data = np.zeros((199, 26))
n = 0
for j in range(18):
    try:
        file = sorted_files[j][0]
        f = open('%s'%(file), 'rb')
        dat=pickle.load(f)
        # Bins=dat['Bins']
        count = np.array(dat['Count'])
        count = count[:199, :]
        # print(np.shape(count))
        Count_data += count
        f.close()
        n+=1
    except:
        pass

In [ ]:
count = dat['Count']
Energy = dat['Energy_range']

In [ ]:
c = 6
plt.plot(Energy_range[0:100], Count_data[0:100, c])
plt.plot(Energy_range[0:100], Count_RAW[0:100, c])
plt.yscale("log")
plt.xlim(0, 1000)

In [ ]:
root_directory="E:/RUN5/Take5/Ba133/Figures/low_Th"  
partial_name = "Efficiency_count"
sorted_files = search_and_sort_files(root_directory, partial_name)
len(sorted_files)

In [ ]:
Count_RAW = np.zeros((199, 26))
n = 0
for j in range(len(sorted_files)):
    try:
        file = sorted_files[j][0]
        f = open('%s'%(file), 'rb')
        dat=pickle.load(f)
        # Bins=dat['Bins']
        count = np.array(dat['Count'])
        count = count.T
        Count_RAW += count
        f.close()
        n+=1
    except:
        pass

In [ ]:
file = sorted_files[0][0]
f = open('%s'%(file), 'rb')
dat=pickle.load(f)
Count = dat['Count']
Energy_range=dat['Energy_range']
np.shape(Count)